# Stacked Denoising AutoEncoder

1. Input Layer : Energy, Contrast, Entropy, Homogeneity, SumAverage, Dissimilarity, AutoCorrelation, Skewness, Kurtosis, Average HU Value
2. Feature 1 : 7
3. Feature 2 : 3
4. Softmax : 2
5. Learning rate : .1 ~ .01
6. Noise factor : 10% on input vector
7. Activation function : sigmoid
8. Cost function : negative log-likelihood

## Three models
1. Feature Extraction : Input -> HL1 -> HL2 -> Decoder -> Weight update
2. Predict Features : Input -> HL1 -> HL2 -> Output
3. Classification : Input of 3 -> HL1 -> HL2 -> Classifier -> Output

In [2]:
import numpy as np
import pandas as pd
from keras.layers import Input, Dense
from keras.models import Model
from keras import optimizers, regularizers
from keras.callbacks import TensorBoard
from keras.utils import to_categorical

# Function for cross validation
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate

Using TensorFlow backend.


In [19]:
# Preparing dataset - total 27039 images : 14294 nodules + 12745 non-nodules
df = pd.read_csv('./Datas/nodules_all_190625.csv', index_col=0)
# select seed to randomize - [0, 3136, 8405, 4242, 5293]
np.random.seed(0)
print(df.shape, '\nDataset loading complete')

(26910, 11) 
Dataset loading complete


In [43]:
# Slice the inputs and labels
# iloc : use integer index for slice
# loc : use label for slice
cols = df.shape[1] - 1
x_train = df.iloc[:, :cols] 
y_train = df.iloc[:, cols]
y_labels_train = to_categorical(y_train)

# add noise to the input data
noise_factor = 0.1
x_train_noisy = x_train * (1 + noise_factor *\
            np.random.binomial(n=1, p=0.1, size=x_train.shape) )
x_train_noisy = np.clip(x_train_noisy, 0., 1.)

In [49]:
## Build Autoencoder
# Input placeholder
input_data = Input(shape=(cols,))

# Hidden Layer 1
hidden_1 = Dense(8, activation='tanh', kernel_initializer='he_normal',
                    )(input_data)

# Hidden Layer 2
hidden_2 = Dense(6, activation='tanh', kernel_initializer='he_normal',
                    )(hidden_1)

# Decoded Layer 1
decoded_1 = Dense(8, activation='sigmoid', kernel_initializer='he_normal')(hidden_2)

# Decoded Layer 2 (reconstructed data)
decoded_2 = Dense(cols, activation='sigmoid', kernel_initializer='he_normal')(decoded_1)

In [50]:
# this model maps an input to its reconstruction
from keras import optimizers

autoencoder_recon = Model(input_data, decoded_2)
ada = optimizers.Adadelta(decay=0)
sgd = optimizers.SGD(lr=0.01)
autoencoder_recon.compile(optimizer=ada, loss='binary_crossentropy')
autoencoder_recon.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (None, 10)                0         
_________________________________________________________________
dense_64 (Dense)             (None, 8)                 88        
_________________________________________________________________
dense_65 (Dense)             (None, 6)                 54        
_________________________________________________________________
dense_66 (Dense)             (None, 8)                 56        
_________________________________________________________________
dense_67 (Dense)             (None, 10)                90        
Total params: 288
Trainable params: 288
Non-trainable params: 0
_________________________________________________________________


In [51]:
autoencoder_recon.fit(x_train_noisy, x_train,
                      epochs=100, batch_size=256,
                      validation_split=0.05,
                      verbose=1,
                      shuffle=True,
                      # callbacks=[TensorBoard(log_dir='./logs/autoencoder_190601-5')]
                     )

Train on 25564 samples, validate on 1346 samples
Epoch 1/100
25564/25564 [==============================] - 2s 66us/step - loss: 0.8136 - val_loss: 0.6570
Epoch 2/100
25564/25564 [==============================] - 0s 11us/step - loss: 0.5891 - val_loss: 0.5567
Epoch 3/100
25564/25564 [==============================] - 0s 11us/step - loss: 0.5259 - val_loss: 0.5190
Epoch 4/100
25564/25564 [==============================] - 0s 11us/step - loss: 0.4979 - val_loss: 0.5001
Epoch 5/100
25564/25564 [==============================] - 0s 11us/step - loss: 0.4828 - val_loss: 0.4895
Epoch 6/100
25564/25564 [==============================] - 0s 11us/step - loss: 0.4741 - val_loss: 0.4835
Epoch 7/100
25564/25564 [==============================] - 0s 11us/step - loss: 0.4688 - val_loss: 0.4804
Epoch 8/100
25564/25564 [==============================] - 0s 11us/step - loss: 0.4656 - val_loss: 0.4786
Epoch 9/100
25564/25564 [==============================] - 0s 11us/step - loss: 0.4638 - val_loss: 0.47

25564/25564 [==============================] - 0s 11us/step - loss: 0.4393 - val_loss: 0.4471
Epoch 78/100
25564/25564 [==============================] - 0s 11us/step - loss: 0.4392 - val_loss: 0.4470
Epoch 79/100
25564/25564 [==============================] - 0s 11us/step - loss: 0.4391 - val_loss: 0.4467
Epoch 80/100
25564/25564 [==============================] - 0s 11us/step - loss: 0.4390 - val_loss: 0.4465
Epoch 81/100
25564/25564 [==============================] - 0s 11us/step - loss: 0.4389 - val_loss: 0.4463
Epoch 82/100
25564/25564 [==============================] - 0s 11us/step - loss: 0.4388 - val_loss: 0.4462
Epoch 83/100
25564/25564 [==============================] - 0s 11us/step - loss: 0.4387 - val_loss: 0.4458
Epoch 84/100
25564/25564 [==============================] - 0s 11us/step - loss: 0.4386 - val_loss: 0.4456
Epoch 85/100
25564/25564 [==============================] - 0s 11us/step - loss: 0.4385 - val_loss: 0.4454
Epoch 86/100
25564/25564 [========================

In [52]:
from keras import Sequential

# Encoder
autoencoder_encoder = Model(input_data, hidden_2)

def create_model(): 
    # create model 
    model = Sequential() 
    model.add(Dense(2, activation='softmax')) 
    # Compile model 
    model.compile(optimizer='adadelta', loss='categorical_crossentropy',
                            metrics=['accuracy'])
    return model

x_feature = autoencoder_encoder.predict(x_train)

# Cross Validation

model = KerasClassifier(build_fn=create_model, epochs=50, batch_size=256)
kfold = KFold(n_splits=5, shuffle=True, random_state=0)
result = cross_validate(model, x_feature, y_train, verbose=0,
                        scoring=['accuracy','precision','recall','f1','roc_auc'], 
                        cv=kfold)

Epoch 1/50
21528/21528 [==============================] - 1s 61us/step - loss: 0.7137 - acc: 0.5321
Epoch 2/50
21528/21528 [==============================] - 0s 7us/step - loss: 0.6613 - acc: 0.6098
Epoch 3/50
21528/21528 [==============================] - 0s 7us/step - loss: 0.6260 - acc: 0.6570
Epoch 4/50
21528/21528 [==============================] - 0s 7us/step - loss: 0.6026 - acc: 0.6843
Epoch 5/50
21528/21528 [==============================] - 0s 7us/step - loss: 0.5876 - acc: 0.7021
Epoch 6/50
21528/21528 [==============================] - 0s 7us/step - loss: 0.5783 - acc: 0.7081
Epoch 7/50
21528/21528 [==============================] - 0s 7us/step - loss: 0.5728 - acc: 0.7135
Epoch 8/50
21528/21528 [==============================] - 0s 7us/step - loss: 0.5697 - acc: 0.7141
Epoch 9/50
21528/21528 [==============================] - 0s 7us/step - loss: 0.5680 - acc: 0.7149
Epoch 10/50
21528/21528 [==============================] - 0s 8us/step - loss: 0.5669 - acc: 0.7153
Epoch 11

21528/21528 [==============================] - 0s 6us/step - loss: 0.5553 - acc: 0.7254
Epoch 34/50
21528/21528 [==============================] - 0s 8us/step - loss: 0.5546 - acc: 0.7260
Epoch 35/50
21528/21528 [==============================] - 0s 7us/step - loss: 0.5539 - acc: 0.7260
Epoch 36/50
21528/21528 [==============================] - 0s 8us/step - loss: 0.5532 - acc: 0.7274
Epoch 37/50
21528/21528 [==============================] - 0s 8us/step - loss: 0.5527 - acc: 0.7280
Epoch 38/50
21528/21528 [==============================] - 0s 7us/step - loss: 0.5520 - acc: 0.7288
Epoch 39/50
21528/21528 [==============================] - 0s 8us/step - loss: 0.5514 - acc: 0.7294
Epoch 40/50
21528/21528 [==============================] - 0s 7us/step - loss: 0.5508 - acc: 0.7295
Epoch 41/50
21528/21528 [==============================] - 0s 7us/step - loss: 0.5502 - acc: 0.7307
Epoch 42/50
21528/21528 [==============================] - 0s 7us/step - loss: 0.5495 - acc: 0.7315
Epoch 43/50


21528/21528 [==============================] - 0s 7us/step - loss: 0.5691 - acc: 0.7110
Epoch 16/50
21528/21528 [==============================] - 0s 7us/step - loss: 0.5681 - acc: 0.7118
Epoch 17/50
21528/21528 [==============================] - 0s 7us/step - loss: 0.5670 - acc: 0.7134
Epoch 18/50
21528/21528 [==============================] - 0s 7us/step - loss: 0.5661 - acc: 0.7143
Epoch 19/50
21528/21528 [==============================] - 0s 7us/step - loss: 0.5652 - acc: 0.7144
Epoch 20/50
21528/21528 [==============================] - 0s 8us/step - loss: 0.5644 - acc: 0.7150
Epoch 21/50
21528/21528 [==============================] - 0s 7us/step - loss: 0.5635 - acc: 0.7160
Epoch 22/50
21528/21528 [==============================] - 0s 8us/step - loss: 0.5627 - acc: 0.7170
Epoch 23/50
21528/21528 [==============================] - 0s 8us/step - loss: 0.5619 - acc: 0.7181
Epoch 24/50
21528/21528 [==============================] - 0s 7us/step - loss: 0.5612 - acc: 0.7186
Epoch 25/50


21528/21528 [==============================] - 0s 8us/step - loss: 0.5470 - acc: 0.7343
Epoch 48/50
21528/21528 [==============================] - 0s 7us/step - loss: 0.5466 - acc: 0.7353
Epoch 49/50
21528/21528 [==============================] - 0s 7us/step - loss: 0.5461 - acc: 0.7361
Epoch 50/50
21528/21528 [==============================] - 0s 7us/step - loss: 0.5458 - acc: 0.7359


## Evaluation Metrics

### 3. Sensitivity(True positive rate, TPR) or Recall
$$ TPR=\frac{TP}{TP+FN} $$

### 4. Precision(Positive predictive value, PPV)
$$ PPV=\frac{TP}{TP+FP} $$

In [53]:
print(result['test_accuracy'], '\n Accuracy Average : ', result['test_accuracy'].mean())
print(result['test_roc_auc'], '\n ROC Average : ', result['test_roc_auc'].mean())
print(result['test_recall'], '\n Recall Average : ', result['test_recall'].mean())
print(result['test_precision'], '\n Precision Average : ', result['test_precision'].mean())
print(result['test_f1'], '\n F-score Average : ', result['test_f1'].mean())

[0.73708658 0.72686734 0.74043107 0.73968785 0.74080268] 
 Accuracy Average :  0.7369751021924935
[0.8082379  0.79395119 0.81005568 0.80089125 0.80815232] 
 ROC Average :  0.804257668348707
[0.78739054 0.7813921  0.80264072 0.79168123 0.7819469 ] 
 Recall Average :  0.7890102987526728
[0.73560209 0.72556025 0.73590315 0.73778502 0.7392905 ] 
 Precision Average :  0.7348282013092242
[0.7606158  0.7524419  0.7678245  0.76378351 0.76002064] 
 F-score Average :  0.7609372703381856


In [32]:
decoded = autoencoder_recon.predict(x_train)
print(decoded)

[[0.03539762 0.03038412 0.13725859 ... 0.2112279  0.08401334 0.5017235 ]
 [0.03661236 0.03129858 0.1386151  ... 0.21433076 0.08619881 0.53268117]
 [0.04142064 0.0347321  0.14365745 ... 0.225766   0.09392393 0.61848646]
 ...
 [0.02695176 0.02376816 0.12559134 ... 0.18867755 0.06719133 0.22075084]
 [0.02653459 0.02353242 0.12515193 ... 0.18733719 0.06687602 0.22435096]
 [0.02631643 0.02332839 0.12464425 ... 0.18687914 0.06629669 0.2146121 ]]


In [ ]:
# 1. Accuracy
print(result, '\n Average Accuracy : ', np.average(result))

In [ ]:
# 2. Area under the ROC-curve (AUC)
predicted = autoencoder_classify.predict(x_train)
prediction_result = np.argmax(predicted, axis=1)

from sklearn.metrics import roc_curve, auc
fpr_keras, tpr_keras, thresholds_keras = roc_curve(y_train.values, predicted[:, 1])

import matplotlib.pyplot as plt
plt.plot(fpr_keras, tpr_keras, label='Keras (area = {:.3f})'.format(auc_keras))
plt.plot([0, 1], [0, 1], 'k--')
plt.legend(loc='best')
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix
TN, FP, FN, TP = confusion_matrix(y_train.values, prediction_result).ravel()

In [ ]:
sensitivity = TP/(TP+FN)
print('Sensitivity : ', sensitivity)

In [ ]:
precision = TP/(TP+FP)
print('Precision : ', precision)

### 5.F-score 
$$ \text{F-score} = \frac{(PPV \times TPR)}{PPV+TPR} $$

In [ ]:
f_score = (precision * sensitivity) / (precision + sensitivity)
print('F-score : ', f_score)

In [ ]:
print(autoencoder_classify.layers)

In [12]:
autoencoder_recon.layers[1].get_weights()

[array([[-0.768768  ,  0.88006055,  0.21781045, -0.0924427 , -0.00164266,
          0.14979221],
        [-0.5339661 ,  0.1496545 , -0.24011269, -0.32375374, -0.11565746,
         -0.418027  ],
        [-0.39518514, -0.31077665, -0.3810753 , -0.23387174, -0.5065482 ,
         -0.08469566],
        [-0.83951217, -0.34855592,  0.8481752 ,  0.13584559, -0.07653016,
         -0.35563365],
        [ 0.03268032, -0.10152446,  0.36609066,  0.48369616, -0.1670611 ,
          0.59071624],
        [-0.43042526,  0.06052483,  0.0949368 , -0.6796002 , -0.12490448,
         -0.3713372 ],
        [-0.04966786, -0.9637751 ,  0.12060661, -0.18130566,  0.28857023,
         -0.49279264],
        [-0.11784459, -0.94431806, -0.0221076 , -0.5024076 ,  0.08656775,
         -0.360977  ],
        [-0.13445432,  0.5711164 , -0.6203959 , -0.6823861 ,  0.07927413,
          0.21285404],
        [ 0.5014185 , -0.29338622, -0.78989136,  0.7650266 , -0.81578267,
          0.3174904 ]], dtype=float32),
 array([ 0.00

In [ ]:
autoencoder_classify.layers[-1].get_weights()

In [ ]:
import matplotlib.pyplot as plt
weight, bias = autoencoder_recon.layers[1].get_weights()

plt.imshow(weight, cmap=plt.cm.autumn)
plt.colorbar()
plt.title('Weights Colormap - 1')

In [ ]:
weight_avg = np.average(weight, axis=1)
plt.plot(range(1,9), weight_avg)
plt.title('Weights Average - 1')
plt.xticks(range(1,9))

In [ ]:
df_weight = pd.DataFrame(weight, 
                         columns=['Feature 1','Feature 2','Feature 3', 'Feature 4',
                                 'Feature 5','Feature 6', 'Feature 7'])

df_weight.to_csv("./Weights_sigmoid_1.csv")

In [ ]:
decoded_all = autoencoder_recon.predict(x_train)
print(decoded_all)
print(x_train)